In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from configuration import get_config

IMG_PATH = r"C:/javier/personal_projects/computer_vision/data/KITTI_object_tracking/images/training/image_02/0000/"
DET_FILE = r"C:/javier/personal_projects/computer_vision/data/KITTI_object_tracking/detections_regionlet/training/det_02/0000.txt"
DATASET = 'KITTI'

In [30]:
config = get_config(DATASET)
dets = pd.read_csv(DET_FILE, header=None, sep=" ")
dets.columns = config['det_cols']
cur_dets = dets.loc[dets['frame'] == 0, :]
cur_dets[cur_dets['score'] > 0]

,frame,track_id,type,truncated,occluded,alpha,left,top,right,bottom,height,width,length,x,y,z,rotation_y,score
18,0,-1,Cyclist,-1,-1,-10,0.0,63.0,80.0,248.0,-1,-1,-1,-1000,-1000,-1000,-10,0.092357
45,0,-1,Car,-1,-1,-10,303.0,167.0,459.0,293.0,-1,-1,-1,-1000,-1000,-1000,-10,14.840359
147,0,-1,Pedestrian,-1,-1,-10,1098.0,156.0,1205.0,324.0,-1,-1,-1,-1000,-1000,-1000,-10,3.611409


In [23]:
import numpy as np
from scipy.optimize import linear_sum_assignment

cost_matrix = np.array([
    [0, 2, 3],
    [2, 1, 0],
    [-100, -10, -20]
])
row_ind, col_ind = linear_sum_assignment(cost_matrix, maximize=True)
row_ind, col_ind 

(array([0, 1, 2], dtype=int64), array([2, 0, 1], dtype=int64))